## **Setting up the Necessities**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
! pip install -q onnxruntime onnx==1.10.0 onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 72.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


The repository for squad contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/squad.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Multi Label Dataset Classifier

/content/drive/My Drive/Multi Label Dataset Classifier


## **Data**

In [ ]:
df = pd.read_csv("dataset_info.csv")
df.head()

,title,url,description,categories
0,Spotify Songs Album,https://www.kaggle.com/datasets/zeesolver/spotfy,"CONTEXT:This dataset provides concise details about music tracks and their performance across various platforms. It includes essential information like track name, artist(s), release date, and presence in popular playlists and charts on platforms like Spotify, Apple Music, Deezer, and Shazam. Additionally, it features metrics such as BPM, key, mode, danceability, valence, energy, acousticness, instrumentalness, and liveness_speechiness, which offer insights into the musical characteristics and appeal of each track. With this data, analysts can evaluate the popularity, genre, and audience e...","Arts and Entertainment,Music,Data Visualization,Exploratory Data Analysis,Websites"
1,Netflix Movies and TV Shows,https://www.kaggle.com/datasets/rahulvyasm/netflix-movies-and-tv-shows,"Netflix stands as a leading force in the realm of media and video streaming. With a staggering array of over 8,000 movies and TV shows accessible on their platform, as of mid-2021, their global subscriber count exceeds 200 million. This tabulated dataset comprehensively catalogues all offerings on Netflix, including vital details such as cast, directors, ratings, release year, duration, and more.Dataset Overview:The Netflix Titles dataset is a comprehensive compilation of movies and TV shows available on Netflix, covering various aspects such as the title type, director, cast, country of p...","Arts and Entertainment,Movies and TV Shows,Data Visualization,Exploratory Data Analysis,Artificial Intelligence"
2,Amazon Bestselling Books,https://www.kaggle.com/datasets/khushikhushikhushi/amazon-bestselling-books,"This dataset contains information about the top-rated books listed on Amazon's Best Sellers in Books section. The data was collected using a web scraping script that iterates through multiple pages of the bestseller list. The dataset includes details such as the book title, author name, rating, and price.Book Name: The title of the book.Author Name: The author(s) of the book.Rating: The rating of the book as given by Amazon customers.Price: The listed price of the book on Amazon.","Computer Science,Beginner,Literature,E-Commerce Services,English"
3,Netflix Stock Price Dataset 🎥🍿🎬📊,https://www.kaggle.com/datasets/mayankanand2701/netflix-stock-price-dataset,"Netflix, Inc. is an American subscription streaming service and production company founded in 1997 by Reed Hastings and Marc Randolph in Scotts Valley, California. Initially, Netflix started as a DVD rental service, pioneering the model of online rentals with no late fees. In 2007, the company transitioned into streaming media, revolutionizing the entertainment industry by offering a vast library of movies and TV shows accessible on-demand. Netflix further expanded its influence by producing original content, beginning with the series ""House of Cards"" in 2013. Today, Netflix is a global po...","Business,Arts and Entertainment,Movies and TV Shows,Tabular,Investing,Data Visualization,Exploratory Data Analysis"
4,Vehicle Population Statistics,https://www.kaggle.com/datasets/prasad22/vehicle-population-statistics,This is a tabular dataset consisting details about 22 types of vehicle population in 13 regions Maharashtra State recorded during the year 2000 to 2018.Below is the list variable dataset contains -Sr No.YearRegionCategoryMotor CyclesScootersMopedCarsJeepsStn. WagonsTaxis meter fitedLuxury /Turist Cabs/,"Tabular,Automobiles and Vehicles,Transportation,Exploratory Data Analysis,Clustering"


In [ ]:
df = df.dropna().reset_index(drop=True)
df.shape

(20904, 4)

In [ ]:
df = df.drop_duplicates(subset='url')
df.shape

(19920, 4)

In [ ]:
df['categories'] = df['categories'].apply(lambda x: x.split(','))

# Convert categories to string type
df['categories'] = df['categories'].astype(str)

categories_list = df.categories.to_list()
category_count = {}
for categories in categories_list:
  category_list = eval(categories)
  for category in category_list:
    if category in category_count.keys():
      category_count[category] += 1
    else:
      category_count[category] = 1
print(f"Number of Categories: {len(category_count)}")
print(category_count)

threshold = int(len(df) * 0.005)
rare_categories = [key for key, value in category_count.items() if value < threshold]
len(rare_categories)

Number of Categories: 437
{'Arts and Entertainment': 915, 'Music': 249, 'Data Visualization': 958, 'Exploratory Data Analysis': 724, 'Websites': 82, 'Movies and TV Shows': 222, 'Artificial Intelligence': 1123, 'Computer Science': 5493, 'Beginner': 1166, 'Literature': 160, 'E-Commerce Services': 127, 'English': 272, 'Business': 1736, 'Tabular': 1312, 'Investing': 225, 'Automobiles and Vehicles': 320, 'Transportation': 130, 'Clustering': 274, 'Games': 242, 'Video Games': 159, 'Education': 5947, 'Universities and Colleges': 1686, 'Categorical': 445, 'Healthcare': 246, 'Global': 157, 'Chemistry': 40, 'Finance': 362, 'Banking': 168, 'Data Cleaning': 434, 'Earth and Nature': 1872, 'Atmospheric Science': 35, 'Advanced': 223, 'Cancer': 149, 'Classification': 3404, 'Medicine': 157, 'Weather and Climate': 77, 'datetime': 24, 'Data Analytics': 935, 'People': 136, 'Feature Engineering': 155, 'Social Networks': 206, 'Deep Learning': 1385, 'Python': 649, 'Neural Networks': 218, 'Economics': 168, 'Re

334

In [ ]:
categories_list = df.categories.to_list()
revised_category_list = []
indices_to_drop = []

for idx, categories in enumerate(categories_list):
  category_list = eval(categories)
  revised_categories = []

  for category in category_list:
    if category not in rare_categories:
      revised_categories.append(category)

  if len(revised_categories) == 0:
    indices_to_drop.append(idx)
  else:
    revised_category_list.append(revised_categories)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_categories'] = revised_category_list
df.shape

(19879, 5)

In [ ]:
revised_categories_list = df.revised_categories.to_list()
revised_category_count = {}
for categories in revised_categories_list:
  category_list = categories
  for category in category_list:
    if category in revised_category_count.keys():
      revised_category_count[category] += 1
    else:
      revised_category_count[category] = 1
print(f"Number of Categories: {len(revised_category_count)}")

Number of Categories: 103


In [ ]:
encode_category_types = { key: idx for idx, (key, value) in enumerate(revised_category_count.items())}
with open("dataset_types_encoded.json", "w") as fp:
  json.dump(encode_category_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_categories_list = df.revised_categories.to_list()

for revised_genres in revised_categories_list:
  categorical_list = [0] * len(encode_category_types)
  for genre in revised_genres:
    genre_type_index = encode_category_types[genre]
    categorical_list[genre_type_index] = 1
  categorical_genre_list.append(categorical_list)

df['dataset_cat_list'] = categorical_genre_list
df.shape

(19879, 6)

In [ ]:
labels = list(encode_category_types.keys())
len(labels), labels[:5]

(103,
 ['Arts and Entertainment',
  'Music',
  'Data Visualization',
  'Exploratory Data Analysis',
  'Movies and TV Shows'])

## **Splitting the dataset**

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(17892, 1987)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,url,description,categories,revised_categories,dataset_cat_list
8983,NBME 4_26 large and largev3,https://www.kaggle.com/datasets/leolu1998/nbme-4-26-large-and-largev3,No description available,"['Pre-Trained Model', 'Standardized Testing']","[Pre-Trained Model, Standardized Testing]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...]"
3000,DSE and CSE datasets,https://www.kaggle.com/datasets/rafiaaaa/dse-and-cse-datasets,No description available,"['Computer Science', 'Tabular']","[Computer Science, Tabular]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
14399,Neural Link Tweets,https://www.kaggle.com/datasets/iabhishekmaurya/neural-link-tweets,ContextJust on this 28th Elon Musk revealed the new Neural Link prototype and also showed it's working.,"['Business', 'News', 'Online Communities', 'NLP', 'Artificial Intelligence']","[Business, News, Online Communities, NLP, Artificial Intelligence]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1529,HR Analytics Classification,https://www.kaggle.com/datasets/bhrt97/hr-analytics-classification,"ContextHR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted....","['Business', 'Software']","[Business, Software]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2905,AutoAIQnA: QNA for Automobile Specs,https://www.kaggle.com/datasets/thejas2002/prompt,"The dataset consists of question and answer pairs specifically tailored for automobiles, encompassing both cars and bikes, along with their make, models, and specifications. These meticulously crafted Q&A pairs were generated utilizing the cutting-edge GPT-4 model, ensuring a comprehensive coverage of diverse topics related to automotive technology. Whether you're an automotive enthusiast, a researcher, or a data scientist, this dataset provides a wealth of information for training and developing AI models capable of understanding and responding to inquiries regarding various automobile ma...","['Automobiles and Vehicles', 'Artificial Intelligence', 'Text Generation']","[Automobiles and Vehicles, Artificial Intelligence]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


## **Fastai and Blurr Inference**

In [ ]:
model_path = "models/dataset-classifier-distilroberta-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("random placeholder")

[{'labels': [],
  'scores': [],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Arts and Entertainment', 'Music', 'Data Visualization', 'Exploratory Data Analysis', 'Movies and TV Shows', 'Artificial Intelligence', 'Computer Science', 'Beginner', 'Literature', 'E-Commerce Services', 'English', 'Business', 'Tabular', 'Investing', 'Automobiles and Vehicles', 'Transportation', 'Clustering', '

In [ ]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

[]

In [ ]:
learner_inf.blurr_predict("It includes information on schools, students, teachers, and educational outcomes across different regions and time periods. The dataset features variables such as school location, student demographics, teacher qualifications, class sizes, school funding, standardized test scores, graduation rates, and post-secondary success indicators. This data can be used for a wide range of analyses, including evaluating the effectiveness of educational policies, identifying trends and disparities in educational attainment, and developing strategies to improve student outcomes. The dataset is an invaluable resource for policymakers, educators, researchers, and anyone interested in understanding and improving education. Please note that all data is anonymized to protect the privacy of individuals. This dataset is regularly updated to reflect the most recent data available in the education sector.")[0]['labels']

['Education']

## **Evaluation**

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['dataset_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_categories = [0] * len(encode_category_types)
  for label in labels:
    pred_categories[encode_category_types[label]] = 1
  preds.append(pred_categories)

preds[0][:20]

  0%|          | 0/1987 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.24253985556615343
F1 Score (Macro) = 0.09043451508218381


## **ONNX**

In [ ]:
model_path = "models/dataset-classifier-distilroberta-stage-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/dataset-classifier-distilroberta.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.0 MB/s eta 0:00:00


In [ ]:
!pip install  onnxruntime

  Using cached onnxruntime-1.18.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (6.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/dataset-classifier-distilroberta.onnx'
quantized_onnx_model_path = 'models/dataset-classifier-distilroberta-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

## **ONNX Inference**

### **Normal ONNX**

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_category_types.keys())

inf_session = rt.InferenceSession('models/dataset-classifier-distilroberta.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_categories = [0] * len(encode_category_types)
  for label in labels:
    pred_categories[encode_category_types[label]] = 1
  preds.append(pred_categories)

  0%|          | 0/1987 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.24225890055926885
F1 Score (Macro) = 0.09076721256064305


### **Quantized ONNX**

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_category_types.keys())

inf_session = rt.InferenceSession('models/dataset-classifier-distilroberta-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_categories = [0] * len(encode_category_types)
  for label in labels:
    pred_categories[encode_category_types[label]] = 1
  preds.append(pred_categories)

  0%|          | 0/1987 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.24514200298953664
F1 Score (Macro) = 0.09169608936342996
